In [7]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [8]:
#export
import torch
from torch import Tensor
import matplotlib.pyplot as plt
import numpy as np

import time
import os
from pathlib import Path
from functools import partial
import re

from fastprogress.fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import format_time

from exp.nb_utils import listify, is_listy, camel2snake
from exp.nb_metrics import AvgLoss, AvgSmoothLoss
from exp.nb_schedules import SchedExp
from exp.nb_opt_utils import set_hyper

In [9]:
#export
class AvgStats():
    def __init__(self, metrics, in_train):
        self.metrics, self.in_train = listify(metrics), in_train
        self.reset()

    def reset(self):
        self.total_loss = 0.
        self.count = 0
        self.total_metrics = [0.] * len(self.metrics)

    @property
    def all_stats(self): return [self.total_loss.item()] + self.total_metrics

    @property
    def avg_stats(self): return [s / self.count for s in self.all_stats]

    def __repr__(self):
        if not self.count:
            return ""
        return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

    def accumulate(self, learner):
        bn = learner.xb.shape[0]
        self.total_loss += learner.loss * bn
        self.count += bn
        for i,m in enumerate(self.metrics):
            self.total_metrics[i] += m(learner.pred, learner.yb) * bn

In [10]:
#export
class Callback():
    _order = 0

    def set_learner(self, learner): self.learner = learner

    def __getattr__(self, k): return getattr(self.learner, k)

    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False

    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')


class CancelTrainException(Exception): pass
class CancelFitException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass
class CancelValidException(Exception): pass

In [11]:
class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)

    def begin_fit(self):
        met_names = ['loss'] + [m.__name__ for m in self.train_stats.metrics]
        names = ['epoch'] + [f'train_{n}' for n in met_names] + [
            f'valid_{n}' for n in met_names] + ['time']
        #Write headers of table
        self.logger(names)

    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        self.start_time = time.time()

    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.learner)

    def after_epoch(self):
        stats = [str(self.epoch)]
        for o in [self.train_stats, self.valid_stats]:
            stats += [f'{v:.6f}' for v in o.avg_stats]
        stats += [format_time(time.time() - self.start_time)]
        #Write row
        self.logger(stats)

In [12]:
#export
class TrainEvalCallback(Callback):
    _order = -1

    def begin_fit(self):
        self.learner.cur_train_epoch_flt = 0.
        self.learner.cur_train_iter = 0

    def begin_train(self):
        self.learner.cur_train_epoch = self.epoch
        self.learner.pct_train = self.epoch / self.epochs
        self.model.train()
        self.learner.in_train = True

    def begin_validate(self):
        self.model.eval()
        self.learner.in_train = False

    def after_batch(self):
        if not self.in_train: return
        self.learner.cur_train_epoch_flt += 1./self.iters
        self.learner.pct_train += 1./(self.iters*self.epochs)
        self.learner.cur_train_iter += 1

In [13]:
#export
class Recorder(Callback):
    _order = 0

    @staticmethod
    def _maybe_item(t):
        t = t.value
        return t.item() if isinstance(t, Tensor) and t.numel() == 1 else t

    def __init__(self, beta=0.98):
        self.loss = AvgLoss()
        self.smooth_loss = AvgSmoothLoss(beta=beta)

    def begin_fit(self):
        self.lrs, self.iters, self.losses, self.values = [], [], [], []
        headers = ["loss"] + [m.name for m in self.metrics]
        train_h = ["train_{}".format(h) for h in headers]
        valid_h = ["valid_{}".format(h) for h in headers]
        headers = ["epoch"] + train_h + valid_h
        headers.append("time")
        self.metric_names = headers
        self.smooth_loss.reset()
#         self.logger(self.metric_names)

    def begin_epoch(self):
        self.cancel_train, self.cancel_valid = False, False
        self.start_epoch = time.time()
        self.log = [getattr(self, 'epoch', 0)]

    def begin_train(self):
        # Reset except smooth_loss
        for m in self._train_mets[1:]:
            m.reset()

    def begin_validate(self):
        for m in self._valid_mets:
            m.reset()

    def after_train(self):
        self.log += map(lambda x: self._maybe_item(x), self._train_mets)

    def after_validate(self):
        self.log += map(lambda x: self._maybe_item(x), self._valid_mets)

    def after_cancel_train(self):
        self.cancel_train = True

    def after_cancel_validate(self):
        self.cancel_validate = True

    def after_epoch(self):
        self.learner.final_record = self.log[1:].copy()
        self.values.append(self.learner.final_record)

        self.log.append(format_time(time.time() - self.start_epoch))
        self.logger(self.log)
        self.iters.append(self.smooth_loss.count)

    def after_batch(self):
        if len(self.yb) == 0: return
        mets = self._train_mets if self.in_train else self._valid_mets
        for met in mets: met.accumulate(self.learner)
        if not self.in_train: return
        self.lrs.append(self.opt.param_groups[-1]['lr'])
        self.losses.append(self.smooth_loss.value)
        self.learner.smooth_loss = self.smooth_loss.value

    def plot_loss(self, skip_start=5, with_valid=True):
        plt.plot(list(range(skip_start, len(self.losses))), self.losses[skip_start:], label='train')
        if with_valid:
            idx = (np.array(self.iters) < skip_start).sum()
            plt.plot(self.iters[idx:], self.values[idx:], label='valid')
            plt.legend()

    def plot_lr_find(self, skip_end=5):
        "Plot the result of an LR Finder test (won't work if you didn't do `learn.lr_find()` before)"
        lrs = self.lrs if skip_end == 0 else self.lrs[:-skip_end]
        losses = self.losses if skip_end == 0 else self.losses[:-skip_end]
        fig, ax = plt.subplots(1,1)
        ax.plot(lrs, losses)
        ax.set_ylabel("Loss")
        ax.set_xlabel("Learning Rate")
        ax.set_xscale('log')

    def plot_sched(self, keys=None, figsize=None):
        assert hasattr(self, 'hps'), '[ERROR] You must run ParamSchedCallback to plot the sched.'
        keys = self.hps.keys() if keys is None else listify(keys)
        rows,cols = (len(keys)+1)//2, min(2, len(keys))
        figsize = figsize or (6*cols,4*rows)
        _, axs = plt.subplots(rows, cols, figsize=figsize)
        axs = axs.flatten() if len(keys) > 1 else listify(axs)
        for p,ax in zip(keys, axs):
            ax.plot(self.hps[p])
            ax.set_ylabel(p)

    @property
    def _train_mets(self):
        if getattr(self, 'cancel_train', False): return []
        return [self.smooth_loss] + self.metrics

    @property
    def _valid_mets(self):
        return [self.loss] + self.metrics

In [14]:
#export
class ParamScheduler(Callback):
    _order = 0

    def __init__(self, scheds):
        self.scheds = scheds

    def _update_val(self, percentage):
        for n, f in self.scheds.items():
            value = f(percentage)
            set_hyper(self.opt, n, value)
#             vs = listify(value)
#             if len(vs) == 1: vs = vs * len(self.opt.param_groups)
#             assert len(vs) == len(self.opt.param_groups), f"Trying to set {len(vs)} values for {n} but there are {len(self.opt.param_groups)} parameter groups."
#             for v,h in zip(vs, self.opt.param_groups):
#                 h[n] = v

    def begin_fit(self):
        self.hps = {p:[] for p in self.scheds.keys()}

    def begin_batch(self):
        self._update_val(self.pct_train)

    def after_batch(self):
        for p in self.scheds.keys():
            self.hps[p].append(self.opt.param_groups[-1][p])

    def after_fit(self):
        if hasattr(self.learner, 'recorder') and hasattr(self, 'hps'):
            self.recorder.hps = self.hps

In [15]:
#export
class ProgressCallback(Callback):
    _order = 1

    def begin_fit(self):
        assert hasattr(self.learner, 'recorder')
        self.mbar = master_bar(list(range(self.epochs)))
        self._write_stats(self.recorder.metric_names)
        self.learner.set_logger(self._write_stats)

    def begin_epoch(self):
        if getattr(self, 'mbar', None): self.mbar.update(self.epoch)

    def begin_train(self):
        self._launch_pbar()

    def begin_validate(self):
        self._launch_pbar()

    def after_train(self):
        self.pbar.on_iter_end()

    def after_validate(self):
        self.pbar.on_iter_end()

    def after_batch(self):
        self.pbar.update(self.iter + 1)
        if hasattr(self, 'smooth_loss'):
            self.pbar.comment = f"{self.smooth_loss:.4f}"

    def after_fit(self):
        if getattr(self, 'mbar', None):
            self.mbar.on_iter_end()
            delattr(self, 'mbar')
        self.learner.logger = print

    def _write_stats(self, log):
        if getattr(self, 'mbar', None):
            self.mbar.write([f"{l:.6f}" if isinstance(l, float) else str(l) for l in log], table=True)

    def _launch_pbar(self):
        self.pbar = progress_bar(self.dl, parent=getattr(self, 'mbar', None), leave=False)
        self.pbar.update(0)

In [16]:
#export
class LRFinder(ParamScheduler):
    _order = 1

    def __init__(self, start_lr=1e-7, end_lr=10, n_iters=100, stop_div=True):
        if is_listy(start_lr):
            self.scheds = {
                'lr': [SchedExp(s, e) for (s, e) in zip(start_lr, end_lr)]
            }
        else:
            self.scheds = {
                'lr': SchedExp(start_lr, end_lr)
            }
        self.n_iters, self.stop_div = n_iters, stop_div

    def begin_fit(self):
        super().begin_fit()
        self.learner.save('_tmp')
        self.best_loss = float('inf')

    def begin_batch(self):
        self._update_val(self.cur_train_iter / self.n_iters)

    def after_batch(self):
        super().after_batch()
        if self.smooth_loss < self.best_loss:
            self.best_loss = self.smooth_loss
        if self.smooth_loss > 4*self.best_loss and self.stop_div:
            raise CancelFitException()
        if self.cur_train_iter >= self.n_iters:
            raise CancelFitException()

    def begin_validate(self):
        raise CancelValidException()

    def after_fit(self):
        self.learner.opt.zero_grad()
        tmp_path = self.model_dir / '_tmp.pth'
        if tmp_path.exists():
            self.learner.load('_tmp')
            os.remove(tmp_path)

In [17]:
#export
class CudaCallback(Callback):
    _order = -1

    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.learner.xb,self.learner.yb = self.xb.cuda(), self.yb.cuda()

In [20]:
!python notebook2script.py callbacks.ipynb

Converted callbacks.ipynb to exp\nb_callbacks.py
